<center><h2 style='color:red'>HuBMAP | Pytorch | ViT for Segmentation [Train]<hr>By Kassem@elcaiseri</h2></center>

This Code depends on: <br>
- The lossses are from **https://github.com/JunMa11/SegLoss**
- The base codes are from here: **https://www.kaggle.com/vineeth1999/hubmap-pytorch-efficientunet-offline**

**Update:**
- It's the official code. https://github.com/Beckschen/TransUNet


- It's the paper of the TransUnet. https://arxiv.org/pdf/2102.04306v1.pdf (Thanks for @luciusk)


## Introduction <br>
Implementation of **self attention mechanisms** for computer vision in PyTorch with einsum and einops. Focused on **segmentation**  self-attention modules.

### Note: This is a baseline code

**Update**
- This code starts from vision 10
- **V15**: - Use DiceBCELoss - increase ViT blocks - 
- **V16**: - Apply Official code.
          - Add more options for loss function.

## Imports

In [ ]:
REDUCED_SET = False

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/DLProjectData/hubmap_train_256.zip" -d "/content"

Streaming output truncated to the last 5000 lines.
  inflating: /content/masks/afa5e8098_0815.png  
  inflating: /content/__MACOSX/masks/._afa5e8098_0815.png  
  inflating: /content/masks/afa5e8098_0801.png  
  inflating: /content/__MACOSX/masks/._afa5e8098_0801.png  
  inflating: /content/masks/b2dc8411c_0193.png  
  inflating: /content/__MACOSX/masks/._b2dc8411c_0193.png  
  inflating: /content/masks/c68fe75ea_0281.png  
  inflating: /content/__MACOSX/masks/._c68fe75ea_0281.png  
  inflating: /content/masks/095bf7a1f_0482.png  
  inflating: /content/__MACOSX/masks/._095bf7a1f_0482.png  
  inflating: /content/masks/cb2d976f4_1151.png  
  inflating: /content/__MACOSX/masks/._cb2d976f4_1151.png  
  inflating: /content/masks/4ef6695ce_1720.png  
  inflating: /content/__MACOSX/masks/._4ef6695ce_1720.png  
  inflating: /content/masks/afa5e8098_0154.png  
  inflating: /content/__MACOSX/masks/._afa5e8098_0154.png  
  inflating: /content/masks/b9a3865fc_0380.png  
  inflating: /content/__MACO

In [ ]:
!pip install albumentations==0.4.6
!pip install segmentation-models-pytorch

     |████████████████████████████████| 122kB 7.5MB/s 
     |████████████████████████████████| 952kB 36.1MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-cp37-none-any.whl size=65163 sha256=f6dff6a0e5d51533c53a5b53978b93599d7c0cfbb39f3373b563c3da4a288705
  Stored in directory: /root/.cache/pip/wheels/c7/f4/89/56d1bee5c421c36c1a951eeb4adcc32fbb82f5344c086efa14
Successfully built albumentations
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12
     |████████████████████████████████| 71kB 5.1MB/s 
     |████████████████████████████████| 245kB 12.0MB/s 
     |████████████████████████████████| 61kB 6.8MB/s 
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-cp37-none-any.whl size=12420 sha256=204278ab7bf506bf05e762ec2665e0147db4eb

In [ ]:
!git clone https://github.com/Beckschen/TransUNet.git
import sys
sys.path.append('/content/TransUNet/')

Cloning into 'TransUNet'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 92 (delta 47), reused 38 (delta 15), pack-reused 0
Unpacking objects: 100% (92/92), done.


In [ ]:
!wget https://storage.googleapis.com/vit_models/imagenet21k/R50%2BViT-B_16.npz
#! pip install self-attention-cv
!pip install einops
!pip install ml_collections
# tu_path = '../input/transunet/TransUNet-main'
# sys.path.append(tu_path)

--2021-05-19 04:24:03--  https://storage.googleapis.com/vit_models/imagenet21k/R50%2BViT-B_16.npz
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 74.125.142.128, 2607:f8b0:400e:c0d::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 461217452 (440M) [application/octet-stream]
Saving to: ‘R50+ViT-B_16.npz’

R50+ViT-B_16.npz    100%[===================>] 439.85M  69.8MB/s    in 7.2s    

2021-05-19 04:24:11 (61.3 MB/s) - ‘R50+ViT-B_16.npz’ saved [461217452/461217452]

     |████████████████████████████████| 92kB 5.6MB/s 


In [ ]:
# import warnings
# warnings.filterwarnings("ignore")

In [ ]:
from sklearn.model_selection import GroupKFold

import torch
import torchvision
from torch import nn
from torch.optim import Adam
from torchvision import transforms
from torch.nn import functional as F
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
#from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau, CosineAnnealingWarmRestarts

import cv2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from scipy.ndimage.interpolation import zoom

import albumentations as A
from albumentations.pytorch import ToTensorV2

import os
import sys
import time
import random

## Hyperparameters

In [ ]:
class CFG:
    data = 256 #512
    debug=False
    apex=False
    print_freq=100
    num_workers=0
    img_size=256 # appropriate input size for encoder 
    scheduler='CosineAnnealingWarmRestarts' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']
    epoch=7 # Change epochs
    criterion= 'DiceBCELoss' #'DiceBCELoss' # ['DiceLoss', 'Hausdorff', 'Lovasz']
    base_model='Unet' # ['Unet']
    encoder = 'vit' # ['attention','efficientnet-b5'] or other encoders from smp
    lr=5e-4
    min_lr=1e-6
    batch_size=16
    weight_decay=1e-6
    gradient_accumulation_steps=1
    seed=2021
    n_fold=5
    trn_fold= 0 #[0, 1, 2, 3, 4]
    train=True
    inference=False
    optimizer = 'Adam'
    T_0=10
    # N=5 
    # M=9
    T_max=10
    #factor=0.2
    #patience=4
    #eps=1e-6
    smoothing=1
    in_channels=3
    vit_blocks=12 #[8, 12]
    vit_linear=1024 #1024
    classes=1
    MODEL_NAME = 'R50-ViT-B_16'


main_dir = '/content'
train_dir = '/content/train'
masks_dir = '/content/masks'

In [ ]:
# from @Iafoss comment
def seed_torch(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    #the following line gives ~10% speedup
    #but may lead to some stochasticity in the results 
    torch.backends.cudnn.benchmark = True # from @Iafoss comment

seed_torch(seed=CFG.seed)

## Dataset

In [ ]:
# train_df = pd.read_csv('/content/train.csv')
# train_df.head()

In [ ]:
# def get_transform(mode='base'):
#     if mode == 'base':
#         base_transform = A.Compose([
#             A.Resize(CFG.img_size, CFG.img_size, p=1.0),
#             A.HorizontalFlip(),
#             A.VerticalFlip(),
#             A.RandomRotate90(),
#             A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=20, p=0.4, 
#                              border_mode=cv2.BORDER_REFLECT),
#             A.OneOf([
#                 A.OpticalDistortion(p=0.4),
#                 A.GridDistortion(p=.1),
#                 A.IAAPiecewiseAffine(p=0.4),
#             ], p=0.3),
#             A.OneOf([
#                 A.HueSaturationValue(10,15,10),
#                 A.CLAHE(clip_limit=3),
#                 A.RandomBrightnessContrast(),            
#             ], p=0.4),
#             ToTensorV2()
#         ], p=1.0)
#         return base_transform
    
#     elif mode == 'rand':
#         rand_transform = A.Compose([
#                 RandAugment(CFG.N, CFG.M),
#                 A.Transpose(p=0.5),
#                 A.VerticalFlip(p=0.5),
#                 A.HorizontalFlip(p=0.5),
#                 A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),
#                 A.Resize(CFG.img_size, CFG.img_size, p=1.0),
#                 A.Normalize(),
#                 ToTensorV2()
#             ])
#         return rand_transform
    
#     elif mode == 'strong':
#         strong_transform = A.Compose([
#                 A.Transpose(p=0.5),
#                 A.VerticalFlip(p=0.5),
#                 A.HorizontalFlip(p=0.5),
#                 A.ElasticTransform(alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03, p=0.5),
#                 A.OneOf([
#                         A.RandomGamma(),
#                         A.GaussNoise()           
#                     ], p=0.5),
#                 A.OneOf([
#                         A.OpticalDistortion(p=0.4),
#                         A.GridDistortion(p=0.2),
#                         A.IAAPiecewiseAffine(p=0.4),
#                     ], p=0.5),
#                 A.OneOf([
#                         A.HueSaturationValue(10,15,10),
#                         A.CLAHE(clip_limit=4),
#                         A.RandomBrightnessContrast(),            
#                     ], p=0.5),

#                 A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),
#                 A.Resize(CFG.img_size, CFG.img_size, p=1.0),
#                 ToTensorV2()
#             ])
#         return strong_transform
    
#     elif mode == 'weak':
#         weak_transform = A.Compose([
#                 A.Resize(CFG.img_size, CFG.img_size, p=0.5),
#                 A.HorizontalFlip(),
#                 A.VerticalFlip(),
#                 A.RandomRotate90(),
#                 A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=15, p=0.4, 
#                                  border_mode=cv2.BORDER_REFLECT),
#                 ToTensorV2()
#             ], p=1.0)
#         return weak_transform
    
#     elif mode == 'valid':
#         val_transform = A.Compose([
#                 A.Resize(CFG.img_size, CFG.img_size, p=1.0),
#                 ToTensorV2()
#             ], p=1.0)
#         return val_transform
    
#     else:
#         print('Unknown mode.')
    

# mean = np.array([0.65459856,0.48386562,0.69428385])
# std = np.array([0.15167958,0.23584107,0.13146145])

In [ ]:
def get_train_transform():
    return A.Compose([
        A.HorizontalFlip(),
            A.OneOf([
                A.RandomContrast(),
                A.RandomGamma(),
                A.RandomBrightness(),
                ], p=0.3),
            A.OneOf([
                A.ElasticTransform(alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
                A.GridDistortion(),
                A.OpticalDistortion(distort_limit=2, shift_limit=0.5),
                ], p=0.3),
            A.ShiftScaleRotate(p=0.2),
            A.Resize(256,256,always_apply=True),
    ],p=1.)

def get_val_transform():
    return A.Compose([
        A.Resize(256,256,always_apply=True),
    ],p=1.)

In [ ]:
# class HuBMAPDataset(Dataset):
#     def __init__(self, main_dir, df, train=True, transform=None):
        
#         self.ids = df.id.values
#         self.fnames = [fname for fname in os.listdir(train_dir) if fname.split('_')[0] in self.ids]

#         self.main_dir = main_dir
#         self.df = df
#         self.train = train
#         self.transform = transform
        
#     def __len__(self):
#         return len(self.fnames)
    
#     def __getitem__(self, idx):
#         fname = self.fnames[idx]
        
#         img = cv2.cvtColor(cv2.imread(os.path.join(main_dir, 'train', fname)), cv2.COLOR_BGR2RGB)
#         mask = cv2.imread(os.path.join(main_dir, 'masks', fname),cv2.IMREAD_GRAYSCALE)
        
#         if self.transform is not None:
#             aug = self.transform(image=img, mask=mask)            
#             img, mask = aug['image'], aug['mask']
                    
#         img = img.type('torch.FloatTensor')
#         img = img/255
#         mask = mask.type('torch.FloatTensor')

#         return img, mask


class HuBMAPDataset(Dataset):
    def __init__(self, ids, phase):
        self.ids = ids
        if phase=='train':
            self.transform = get_train_transform()
        else:
            self.transform = get_val_transform()
        
    def __getitem__(self, idx):
        name = self.ids[idx]
        img = cv2.imread(os.path.join(train_dir, name)).astype("float32")[:,:,::-1] # changed path
        img /= 255.
        mask = cv2.imread(os.path.join(masks_dir, name))[:,:,0:1] # changed path

        transformed = self.transform(image=img, mask=mask)
        img = transformed['image']
        mask = transformed['mask']
        img = img.transpose(2,0,1).astype('float32')
        mask = mask.transpose(2,0,1).astype('float32')
        return img, mask

    def __len__(self):
        return len(self.ids)
          

## Visualize augmented data

In [ ]:
# def vis_aug_data(dataset, length=6):
#     plt.figure(figsize=(15,10))
#     N = length // 2
#     for i in range(length):
#         image, mask = train_dataset[i]
#         plt.subplot(3,4,2*i+1)
#         plt.imshow(np.transpose((image), (1,2,0)))
#         plt.axis('off')
#         plt.subplot(3,4,2*i+2)
#         plt.imshow(mask)
#         plt.axis('off')

In [ ]:
# # base augmentation
# train_dataset = HuBMAPDataset(main_dir, train_df, train=True, transform=get_transform('base'))

# vis_aug_data(train_dataset, 6)

In [ ]:
# # weak augmentation
# train_dataset = HuBMAPDataset(main_dir, train_df, train=True, transform=get_transform('weak'))

# vis_aug_data(train_dataset, 6)

In [ ]:
# # strong augmentation
# train_dataset = HuBMAPDataset(main_dir, train_df, train=True, transform=get_transform('strong'))

# vis_aug_data(train_dataset, 6)

In [ ]:
# # valid augmentation
# train_dataset = HuBMAPDataset(main_dir, train_df, train=True, transform=get_transform('valid'))

# vis_aug_data(train_dataset, 6)

## DataLoader

In [ ]:
# if CFG.data==512:
#     directory_list = os.listdir('../input/hubmap-512x512/train')
# elif CFG.data==256:
#     directory_list = os.listdir('../input/hubmap-256x256/train')

# directory_list = [fnames.split('_')[0] for fnames in directory_list]
# dir_df = pd.DataFrame(directory_list, columns=['id'])
# dir_df


directory_list = os.listdir(train_dir) # changed path
dir_df = pd.DataFrame(directory_list, columns=['Image_Paths'])

## Model

In [ ]:
from networks.vit_seg_modeling import VisionTransformer as ViT_seg
from networks.vit_seg_modeling import CONFIGS as CONFIGS_ViT_seg

config_vit = CONFIGS_ViT_seg[CFG.MODEL_NAME]
config_vit.n_classes = 1
config_vit.n_skip = 3
config_vit.pretrained_path = './R50+ViT-B_16.npz'
config_vit.transformer.dropout_rate = 0.2
config_vit.transformer.mlp_dim = 3072
config_vit.transformer.num_heads = 4
config_vit.transformer.num_layers = 8

config_vit

activation: softmax
classifier: seg
decoder_channels: !!python/tuple
- 256
- 128
- 64
- 16
hidden_size: 768
n_classes: 1
n_skip: 3
patch_size: 16
patches:
  grid: !!python/tuple
  - 16
  - 16
  size: !!python/tuple
  - 16
  - 16
pretrained_path: ./R50+ViT-B_16.npz
representation_size: null
resnet:
  num_layers: !!python/tuple
  - 3
  - 4
  - 9
  width_factor: 1
resnet_pretrained_path: null
skip_channels:
- 512
- 256
- 64
- 16
transformer:
  attention_dropout_rate: 0.0
  dropout_rate: 0.2
  mlp_dim: 3072
  num_heads: 4
  num_layers: 8

In [ ]:
class ViTHuBMAP(nn.Module):
    def __init__(self, configs=config_vit):
        super(ViTHuBMAP, self).__init__()
        
        self.model = ViT_seg(configs, img_size=CFG.img_size, num_classes=CFG.classes)
        self.model.load_from(weights=np.load(configs.pretrained_path))

    
    def forward(self, x):
        img_segs = self.model(x)
        
        return img_segs

## Loss Function

In [ ]:
# sys.path.append('../input/segloss/Segmentationloss/SegLoss-master')
# from losses_pytorch.hausdorff import HausdorffDTLoss
# from losses_pytorch.lovasz_loss import LovaszSoftmax
# from losses_pytorch.focal_loss import FocalLoss

In [ ]:
class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=CFG.smoothing):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        
        return dice

def getDiceScore(y_hat, y, smooth=1e-8):
    assert(y_hat.shape == y.shape)

    #comment out if your model contains a sigmoid or equivalent activation layer
    y_hat = torch.sigmoid(y_hat)

    #flatten label and prediction tensors
    y_hat = y_hat.view(-1)
    y_hat = (y_hat >= 0.5).float()
    y = y.view(-1)

    intersection = (y_hat * y).sum()                            
    dice = (2.*intersection + smooth)/(y_hat.sum() + y.sum() + smooth)  

    return dice.item()   
    
    
class DiceBCELoss(nn.Module):
    # Formula Given above.
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=CFG.smoothing):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).mean()                            
        dice_loss = 1 - (2.*intersection + smooth)/(inputs.mean() + targets.mean() + smooth)  
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        Dice_BCE = BCE + dice_loss
        
        return Dice_BCE.mean()
    
    
# class Hausdorff_loss(nn.Module):
#     def __init__(self):
#         super(Hausdorff_loss, self).__init__()
        
#     def forward(self, inputs, targets):
#         return HausdorffDTLoss()(inputs, targets)
    
# class FocalDLoss(nn.Module):
#     def __init__(self):
#         super(FocalDLoss, self).__init__()
        
#     def forward(self, inputs, targets):
#         return FocalLoss()(inputs, targets)
    
    
# class Lovasz_loss(nn.Module):
#     def __init__(self):
#         super(Lovasz_loss, self).__init__()
        
#     def forward(self, inputs, targets):
#         return LovaszSoftmax()(inputs, targets)

In [ ]:
if CFG.criterion == 'DiceBCELoss':
    criterion = DiceBCELoss()
elif CFG.criterion == 'DiceLoss':
    criterion = DiceLoss()
else:
    raise Exception("SLDKFJSDF")
# elif CFG.criterion == 'FocalLoss':
#     criterion = FocalDLoss()
# elif CFG.criterion == 'Hausdorff':
#     criterion = Hausdorff_loss()
# elif CFG.criterion == 'Lovasz':
#     criterion = Lovasz_loss()

## Train Function

In [ ]:
def HuBMAPLoss(images, targets, model, device, loss_func=criterion):
    model.to(device)
    images = images.to(device)
    targets = targets.to(device)
    outputs = model(images)
    loss_func = loss_func
    loss = loss_func(outputs, targets)
    return loss, outputs

In [ ]:
def timer(msg, reset = False):
    now = time.time();
    if not hasattr(timer, "start") or reset:
        timer.start = now;
        diff = 0;
    else:
        diff = now - timer.start;
        
    print(f"{msg} | {diff} seconds");

In [ ]:
train_loss_per_epoch = []
train_score_per_epoch = []

valid_loss_per_epoch = []
valid_score_per_epoch = []

# 1st entry is the train loss, 2nd entry is the valid loss, 3rd entry is the train dice score, 4th entry is the valid dice score
fold_dict = {}

In [ ]:
def train_one_epoch(epoch, model, device, optimizer, scheduler, trainloader):
    model.train()
    t = time.time()
    total_loss = 0
    dice_epoch = []
    for step, (images, targets) in enumerate(trainloader):
        # run model and get loss
        loss, outputs = HuBMAPLoss(images, targets, model, device)
        loss.backward()
        if ((step+1)%4==0 or (step+1)==len(trainloader)):
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        loss = loss.detach().item()
        total_loss += loss

        # get dice score
        dice_batch = getDiceScore(outputs, targets.to(device))
        dice_epoch.append(dice_batch)
        
        # if ((step+1)%10==0 or (step+1)==len(trainloader)):
        #     print(
        #             f'epoch {epoch} train step {step+1}/{len(trainloader)}, ' + \
        #             f'loss: {total_loss/len(trainloader):.4f}, ' + \
        #             f'time: {(time.time() - t):.4f}', end= '\r' if (step + 1) != len(trainloader) else '\n'
        #         )

    # print stuff
    print(
            f'epoch {epoch}, ' + \
            f'loss: {total_loss/len(trainloader):.4f}, ' + \
            f'time: {(time.time() - t):.4f}, '  + \
            f'dice: {np.mean(dice_epoch)}', end='\n'
        )
    
    return total_loss/len(trainloader), np.mean(dice_epoch)
            
        
def valid_one_epoch(epoch, model, device, optimizer, scheduler, validloader):
    model.eval()
    t = time.time()
    total_loss = 0
    dice_epoch = []
    for step, (images, targets) in enumerate(validloader):
        # run model and get loss
        loss, outputs = HuBMAPLoss(images, targets, model, device)
        loss = loss.detach().item()
        total_loss += loss
        
        if ((step+1)%4==0 or (step+1)==len(validloader)):
            scheduler.step(total_loss/len(validloader))

        # get dice score
        dice_batch = getDiceScore(outputs, targets.to(device))
        dice_epoch.append(dice_batch)
        
        # if ((step+1)%10==0 or (step+1)==len(validloader)):
        #     print(
        #             f'**epoch {epoch} trainz step {step+1}/{len(validloader)}, ' + \
        #             f'loss: {total_loss/len(validloader):.4f}, ' + \
        #             f'time: {(time.time() - t):.4f}', end= '\r' if (step + 1) != len(validloader) else '\n'
        #         )

    # print stuff       
    print(
            f'epoch {epoch}, ' + \
            f'loss: {total_loss/len(trainloader):.4f}, ' + \
            f'time: {(time.time() - t):.4f}, '  + \
            f'dice: {np.mean(dice_epoch)}', end='\n'
        )
    return total_loss/len(validloader), np.mean(dice_epoch)

## Creating Folds Column

In [ ]:
FOLDS = CFG.n_fold
gkf = GroupKFold(FOLDS)
dir_df['Folds'] = 0
for fold, (tr_idx, val_idx) in enumerate(gkf.split(dir_df, groups=dir_df[dir_df.columns[0]].values)):
    dir_df.loc[val_idx, 'Folds'] = fold
    
dir_df

,Image_Paths,Folds
0,afa5e8098_0306.png,4
1,cb2d976f4_1135.png,0
2,1e2425f28_0071.png,2
3,1e2425f28_0476.png,4
4,cb2d976f4_0643.png,2
...,...,...
9575,4ef6695ce_1341.png,1
9576,cb2d976f4_0766.png,1
9577,afa5e8098_1418.png,1
9578,0486052bb_0646.png,3


## The Real Training

In [ ]:
# def prepare_train_valid_dataloader(df, fold):
#     train_ids = df[~df.Folds.isin(fold)]
#     val_ids = df[df.Folds.isin(fold)]
    
#     train_ds = HuBMAPDataset(main_dir, train_ids, train=True, transform=get_transform('base'))
#     val_ds = HuBMAPDataset(main_dir, val_ids, train=True, transform=get_transform('valid'))
    
#     train_loader = DataLoader(train_ds, batch_size=CFG.batch_size, pin_memory=True, shuffle=True, num_workers=CFG.num_workers)
#     val_loader = DataLoader(val_ds, batch_size=CFG.batch_size, pin_memory=True, shuffle=False, num_workers=CFG.num_workers)
    
#     return train_loader, val_loader



def prepare_train_valid_dataloader(df, fold):
    train_ids = df.loc[~df.Folds.isin(fold), "Image_Paths"].values
    val_ids = df.loc[df.Folds.isin(fold), "Image_Paths"].values

    train_ds = HuBMAPDataset(train_ids, "train")
    val_ds = HuBMAPDataset(val_ids, "val")

    train_loader = DataLoader(train_ds, batch_size=CFG.batch_size, pin_memory=True, shuffle=True, num_workers=CFG.num_workers)
    val_loader = DataLoader(val_ds, batch_size=CFG.batch_size, pin_memory=True, shuffle=False, num_workers=CFG.num_workers)

    return train_loader, val_loader

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = ViTHuBMAP().to(device)
optimizer = Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay, amsgrad=False)

# scheduler setting
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, gamma=0.1, step_size=1)
# if CFG.scheduler == 'CosineAnnealingWarmRestarts':
#     scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=CFG.T_0, T_mult=1, eta_min=CFG.min_lr, last_epoch=-1)
# elif CFG.scheduler == 'ReduceLROnPlateau':
#     scheduler = ReduceLROnPlateauReduceLROnPlateau(optimizer, mode='min', factor=CFG.factor, patience=CFG.patience, verbose=True, eps=CFG.eps)
# elif CFG.scheduler == 'CosineAnnealingLR':
#     scheduler = CosineAnnealingLR(optimizer, T_max=CFG.T_max, eta_min=CFG.min_lr, last_epoch=-1)

load_pretrained: grid-size from 14 to 16


In [ ]:
timer("starting training", reset = True);
for fold, (tr_idx, val_idx) in enumerate(gkf.split(dir_df, groups=dir_df[dir_df.columns[0]].values)):
    timer(f"{fold} fold loop")
    # if fold>1: # think this is just doing 1 cross validation ...
    #     break
        
    trainloader, validloader = prepare_train_valid_dataloader(dir_df, [fold])
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    model = ViTHuBMAP().to(device)
    optimizer = Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay, amsgrad=False)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, gamma=0.1, step_size=1)
    
    
    for epoch in range(CFG.epoch):
        # train
        train_loss, train_score = train_one_epoch(epoch, model, device, optimizer, scheduler, trainloader)
        train_loss_per_epoch.append(train_loss)
        train_score_per_epoch.append(train_score)

        # validate
        with torch.no_grad():
            valid_loss, valid_score = valid_one_epoch(epoch, model, device, optimizer, scheduler, validloader)
            valid_loss_per_epoch.append(valid_loss)
            valid_score_per_epoch.append(valid_score)
        
    fold_dict[fold] = (train_loss_per_epoch, valid_loss_per_epoch, train_score_per_epoch, valid_score_per_epoch)

    train_loss_per_epoch = []
    train_score_per_epoch = []
    valid_loss_per_epoch = []
    valid_score_per_epoch = []
    
    # save state per fold
    torch.save(model.state_dict(), f'FOLD-{fold}-model.pth')

    timer(f"{fold} done fold")

starting training | 0 seconds
0 fold loop | 0.03522944450378418 seconds
load_pretrained: grid-size from 14 to 16


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


epoch 0, loss: 0.9557, time: 308.8438, dice: 0.09010562495647352


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:154: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


epoch 0, loss: 0.2391, time: 27.1845, dice: 0.10759462454977135
epoch 1, loss: 0.3434, time: 304.7429, dice: 0.7138292494391356
epoch 1, loss: 0.0401, time: 26.4072, dice: 0.7341241578261138
epoch 2, loss: 0.0983, time: 304.8608, dice: 0.7930590311893629
epoch 2, loss: 0.0569, time: 26.3798, dice: 0.5304427323242055
epoch 3, loss: 0.0627, time: 304.3905, dice: 0.8226593534534115
epoch 3, loss: 0.0116, time: 26.3885, dice: 0.8761151055496813
epoch 4, loss: 0.0447, time: 304.3704, dice: 0.8629337080709685
epoch 4, loss: 0.0099, time: 26.4308, dice: 0.8781569202742655
epoch 5, loss: 0.0388, time: 303.7617, dice: 0.8678979515594114
epoch 5, loss: 0.0089, time: 26.4525, dice: 0.8833972555895646
epoch 6, loss: 0.0341, time: 304.7973, dice: 0.8787463760326499
epoch 6, loss: 0.0071, time: 26.4298, dice: 0.898419985672025
0 done fold | 2323.9572751522064 seconds
1 fold loop | 2323.9579515457153 seconds
load_pretrained: grid-size from 14 to 16
epoch 0, loss: 1.2530, time: 304.5356, dice: 0.06101

In [ ]:
# print(f'Training Loop [{CFG.trn_fold}]...')
# for fold, (tr_idx, val_idx) in enumerate(gkf.split(dir_df, groups=dir_df[dir_df.columns[0]].values)):
#     # if fold != CFG.trn_fold: # Train only one fold
#     #     continue 

#     trainloader, validloader = prepare_train_valid_dataloader(dir_df, [fold])

#     for epoch in range(CFG.epoch):
#         train_one_epoch(epoch, model, device, optimizer, scheduler, trainloader)
#         with torch.no_grad():
#             valid_one_epoch(epoch, model, device, optimizer, scheduler, validloader)
        
#         #torch.save(model.state_dict(),f'FOLD-{fold}-EPOCH-{epoch}-model.pth')
        
#     torch.save(model.state_dict(),f'FOLD-{fold}-model.pth')
    
#     #del model, optimizer, scheduler, trainloader, validloader

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# loss score for epochs only

for i in range(CFG.n_fold):
    plt.plot(fold_dict[i][0], label='train', color = 'orange')
    plt.plot(fold_dict[i][1], label='valid', color='blue')

    plt.xlabel('Epoch')
    plt.ylabel('DiceBCE Loss')

    plt.title("DiceBCE Loss vs Epoch for Validation Set {0}".format(i))
    plt.legend()
    plt.show()

In [ ]:
# dice score for epochs only

for i in range(CFG.n_fold):
    plt.plot(fold_dict[i][2], label='train', color = 'orange')
    plt.plot(fold_dict[i][3], label='valid', color='blue')

    plt.xlabel('Epoch')
    plt.ylabel('Average Dice Score')

    plt.title("Dice Score vs Epoch for Validation Set {0}".format(i))
    plt.legend()
    plt.show()

## Plot the results 

In [ ]:
# trainloader, validloader = prepare_train_valid_dataloader(dir_df, [4])

In [ ]:
# def plot_result(validloader, n_sample=4):
#     img, mask = next(iter(validloader))
#     model.load_state_dict(torch.load(f'./FOLD-{CFG.trn_fold}-model.pth'))
#     pred_mask = model(img.to(device)).squeeze(1).cpu().detach()
    
#     N = n_sample // 2
#     assert (N < CFG.batch_size)
#     plt.figure(figsize=(15, 20))
#     for i in range(n_sample):
#         plt.subplot(N, 4, 2*i+1)
#         plt.imshow(np.transpose(img[i], (1,2,0)))
#         plt.axis('off')
        
#         plt.subplot(N, 4, 2*i+2)
#         plt.imshow(pred_mask[i])
#         plt.axis('off')

In [ ]:
# with torch.no_grad():
#     plot_result(validloader)

## What you should do to improve the results:
- Use data normalization
- Use anthor implementation for TransUnet (Applied in V16)
- Reduce the probability of the augmentations
- Play with visionTrans hyperparameters

<CENTER><h3><span style='color:red'> UPVOTE </span> If you liked</h3>
<h4>Code still under modifications, Stay tuned</h4></CENTER>